<img src="Tutorial-DirectStitch-figs/Chameleon-FullColor-lg_4CCULFA.jpg" >

# Tutorial: Advanced uses of Chameleon DirectStitch Connections 

- **Estimated time**: ?? minutes
- **Requirements**:

     - This tutorial assumes you have a Chameleon account and basic experience logging into and using Chameleon (see: https://chameleoncloud.readthedocs.io/en/latest/getting-started/index.html)
     - Understanding of Chameleon networks and isolated VLANs. (see: https://chameleoncloud.readthedocs.io/en/latest/technical/networks.html)
     - Basic understanding of the CLI (see https://chameleoncloud.readthedocs.io/en/latest/technical/cli.html)
     - Have a GENI account and SSL Certificate (see http://www.exogeni.net/2015/09/exogeni-getting-started-tutorial/)
     - Have reviewed the basic DirectStitch tutorial (see https://jupyter.chameleoncloud.org/hub/user-redirect/lab/tree/git/notebooks/tutorials/networking/Tutorial-DirectStitch.ipynb)




The Chameleon DirectStitch capability provides direct OSI Layer 2 networking connections between Chameleon resources and external facilities. Chameleon DirectStitch is similar services offered by public cloud providers such as AWS Direct Connect, Google Cloud Dedicated Interconnect, and Microsoft Azure ExpressRoute. The basic Chameleon DirectStitch tutorial shows how to create private networks that can be connected with external circuits to remote facilities (including networks in the other Chameleon site). 

Method presented in the basic tutorial is limited to a single externally stitched VLAN and it is difficult to modify the network independently from the compute nodes. The more advance method presented in this tutorial shows how separate management of the stitched links the compute nodes.  Separate management allows for multiple stitched links to be dynamically added and removed after the experiment has been instantiated. The flexibility  provided by this method makes it the recommended method for all DirectStitch experiments. 


During this tutorial we will:

- Step 1: Create a Chameleon Isolated Network with compute hosts.
- Step 2: Dynamically  add/remove DirectStitch connections.


## Tutorial

In this tutorial, you will use Chameleon DirectStitch ports to connect to connect an isolated tenant network to one or more external campus facilities or other resource on ExoGENI.


### Set Variables

In [ ]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
#export OS_PROJECT_NAME='ExoGENI@Chameleon'
export OS_PROJECT_NAME="CH-816532"
export OS_PROJECT_DOMAIN_NAME=chameleon

# Set chameleon keypair name and path to the private ssh key
export SSH_KEY_NAME=${USER}-jupyter
export SSH_PRIVATE_KEY=${HOME}/work/pruth-chameleon-jupyter

export NETWORK_SUBNET="192.168.100.0/24"

export RESOURCE_PREFIX="$USER-test2-$(date +%b%d)"

export GENI_PEM_FILE="~/work/geni-pruth.pem"

echo $SSH_KEY_NAME

## Step 1: Create a Local Chameleon Isolated Network

The first step is to create an isolated network. This network will be the base network used by the compute nodes. The base network should be on the "physnet1" provided (i.e. is should not be an externally stitchable network).  However, it is important to include a "VSwitchName". Later you will use this name to add externally stitchable ports to the isolated network. Optionally, you can specify an OpenFlow controller and control the traffic flowing through this switch.

<img src="Tutorial-DirectStitch-figs/AdvancedDirectStitch1.png" width="500">

### Step 1a: Create the Network

#### Set network resource names

In [ ]:
LOCAL_NET_LEASE_NAME=${RESOURCE_PREFIX}"-LocalNet"

# Set the names for the network, subnet, router, and switch. 
NETWORK_NAME=${RESOURCE_PREFIX}"-Network"
SUBNET_NAME=${RESOURCE_PREFIX}"-Subnet"
ROUTER_NAME=${RESOURCE_PREFIX}"-Router"

PROVIDER='physnet1'
SWITCH_NAME=${RESOURCE_PREFIX}'Switch'

In [ ]:
# Create Lease for local network on the physnet1 provider with a unique VSwitchName
blazar lease-create \
   --reservation resource_type=network,network_name=${NETWORK_NAME},network_description="VSwitchName=${SWITCH_NAME}",resource_properties="[\"==\",\"\$physical_network\",\"$PROVIDER\"]" \
   ${LOCAL_NET_LEASE_NAME}

#### Step 1b: Add a subnet and router to the network

In [ ]:
echo Creating Subnet
openstack subnet create --max-width 80 \
                        --subnet-range ${NETWORK_SUBNET} \
                        --dhcp \
                        --network ${NETWORK_NAME} \
                        ${SUBNET_NAME}
                        
echo Creating Router
openstack router create --max-width 80 ${ROUTER_NAME}

echo Linking router to subnet
openstack router add subnet ${ROUTER_NAME} ${SUBNET_NAME}

echo Linking router to external gateway
openstack router set --external-gateway public ${ROUTER_NAME}

echo Network ${NETWORK_NAME} is ready for nodes!

#### Step 1c: Launch servers connected to the new network

At this point your OpenFlow network is ready for compute nodes. You can add nodes using the CLI commands below or by any other method you are comfortable with. 

In [ ]:
START_DATE=`date -d "+2 min" +'%F %H:%M'`
END_DATE=`date -d "+1 day" +'%F %H:%M'`


NODE_TYPE="compute_haswell"
SERVER_LEASE_NAME="${RESOURCE_PREFIX}-Servers" 

#MIN=2
#MAX=2

blazar lease-create \
      --physical-reservation min=${MIN},max=${MAX},resource_properties="[\"==\",\"\$node_type\",\"compute_haswell\"]" \
      --start-date "${START_DATE}" \
      --end-date "${END_DATE}" \
      ${SERVER_LEASE_NAME}

# Get the reservation ID

RESERVATION_ID=$(blazar lease-show  -f json ${SERVER_LEASE_NAME} | jq -r .reservations | jq -r .id)
LEASE_ID=$(blazar lease-show  -f json ${SERVER_LEASE_NAME} | jq -r .id)


In [ ]:
NODE_NAME=${RESOURCE_PREFIX}-node

echo $SSH_KEY_NAME

echo Creating servers... This will take several minutes! 
openstack server create --max-width 80 \
                        --flavor "baremetal" \
                        --image "CC-CentOS7" \
                        --key-name ${SSH_KEY_NAME} \
                        --hint reservation=${RESERVATION_ID} \
                        --security-group default  \
                        --nic net-id=${NETWORK_NAME} \
                        ${NODE_NAME}

## Step 2: Add Stitched Links to the Existing Network Switch

DirectStitch links are network that must be reserved using the reservation system.  When you create this reservation, you must specify the "exogeni" provider network to get an externally stitchable link. It is important to specify the "VSwitchName" of the base network you created in step 1. This will instruct Chameleon to add this new stitched link to the existing network. If you do not specify the name, the Chameleon will create a new isolated network as in the basic DirectStitch tutorial.

<img src="Tutorial-DirectStitch-figs/AdvancedDirectStitch2.png" width="500">


#### Step 2a: Create a lease for the DirectStitch network

This step can be re-run with another network and reservation name in order to create multiple DirectStitch links connected to the same base network.

In [ ]:
START_DATE=`date -d "+2 min" +'%F %H:%M'`
END_DATE=`date -d "+1 day" +'%F %H:%M'`

STITCH_NETWORK_NAME=${RESOURCE_PREFIX}'-exogeni-stitch'
STITCH_RESERVATION_NAME=${RESOURCE_PREFIX}'-exogeni-stitch'
PROVIDER='exogeni'

blazar lease-create \
   --reservation resource_type="network",network_name="${STITCH_NETWORK_NAME}",network_description="VSwitchName=${SWITCH_NAME}",resource_properties="[\"==\",\"\$physical_network\",\"$PROVIDER\"]" \
   --start-date "${START_DATE}" \
   --end-date "${END_DATE}" \
   ${STITCH_RESERVATION_NAME}


#### Step 2b: Get the DirectStitch VLAN Tag

You will need to get the DirectStitch VLAN Tag in order to connect an external facility. If you add multiple DirectStitch ports to a single network, each port will have a unique VLAN tag that can be found using its specified network name.

Note: this will not be successful until the network is created. You may need to re-run this command until the VLAN tag is found.

In [ ]:
DIRECTSTITCH_VLAN=`openstack network show -c provider:segmentation_id -f value ${STITCH_NETWORK_NAME}`
echo DirectStitch VLAN: $DIRECTSTITCH_VLAN 

## Step 3: Use ExoGENI to Connect the Ports

This can be accomplished  with 'xoStitch' tool or any other mechanism for requesting ExoGENI sllices. See the basic DirectStitch tutorial for details.


## Step 4: Delete DirectStitch Links

In order to delete DirectStitch ports simply delete the lease that created them. The method presented in this tutorial allows for DirectStitch ports be added and removed throughout the experiment.

In [ ]:
blazar lease-delete $STITCH_RESERVATION_NAME